In [5]:
import json
import random
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
nltk.download("stopwords")
from keras.layers import Embedding
import pymorphy2
#morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\v.mikheev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\v.mikheev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\v.mikheev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
!pip install tensorflow

   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/385.2 MB ? eta -:--:--
   ---------------------------------------- 1.6/385.2 MB 6.4 MB/s eta 0:01:00
   ---------------------------------------- 3.9/385.2 MB 7.8 MB/s eta 0:00:49
    --------------------------------------- 7.3/385.2 MB 10.3 MB/s eta 0:00:37
    --------------------------------------- 8.1/385.2 MB 8.8 MB/s eta 0:00:43
   - -------------------------------------- 10.0/385.2 MB 9.0 MB/s eta 0:00:42
   - -------------------------------------- 11.8/385.2 MB 8.9 MB/s eta 0:00:42
   - -------------------------------------- 13.6/385.2 MB 8.9 MB/s eta 0:00:42
   - -------------------------------------- 15.7/385.2 MB 9.0 MB/s eta 0:00:42
   - -------------------------------------- 17.6/385.2 MB 9.0 MB/s eta 0:00:41
   -- ------------------------------------- 19.7/385.2 MB 9.1 MB/s eta 0:00:41
   -- ------------------------------------- 21.8/385.2 MB 9.1 MB/s eta 

In [1]:
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)

In [7]:
pymorphy2_311_hotfix()
morph = pymorphy2.MorphAnalyzer()

In [11]:
data_file = open('C:/Users/v.mikheev/Downloads/intentsbars.json',encoding='utf-8').read()

In [13]:
def lemmatizem(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [15]:
from nltk.corpus import stopwords
intents = json.loads(data_file,strict=False)
russian_stopwords = stopwords.words("russian")
words=[]
classes = []
documents = []
ignore_words = ['?', '!',',']+ russian_stopwords

In [17]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = lemmatizem(pattern)
        words.extend(w)

        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(words), "unique lemmatized words")
print (len(classes), "classes", classes)

15 documents
28 unique lemmatized words
6 classes ['DWH', 'ETL', 'Задачи Alpha BI', 'Метод пузырька', 'Регламенты и инструкции общего характера', 'Туалет']


In [19]:
training = []
output_empty = [0] * len(classes)
for doc in documents:

    # initializing bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.asarray(training, dtype="object")
#training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

['alpha', 'bi', 'dwh', 'etl', 'python', 'барс', 'груп', 'задача', 'инструкция', 'комната', 'метод', 'нужный', 'общий', 'предложить', 'простой', 'процесс', 'пузырёк', 'работа', 'рассказать', 'регламент', 'режим', 'решать', 'сортировка', 'туалет', 'уборный', 'узнать', 'характер', 'хотеть']


In [21]:
training = []
output_empty = [0] * len(classes)
for doc in documents:

    # initializing bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.asarray(training, dtype="object")
#training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [23]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(64,input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\v.mikheev\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.1250 - loss: 1.8861  
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1667 - loss: 1.8392 
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 1.8654 
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2000 - loss: 1.8119 
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3333 - loss: 1.6848 
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 1.8018 
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 1.6632 
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2750 - loss: 1.6132     
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4083 - loss: 1.5134 
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3667 - loss: 1.6832 
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7083 - loss: 1.5590 
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5833

In [27]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [29]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [31]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

In [33]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            print(result)
            break
    return result

In [35]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)

In [37]:
query = 'Что такое? DWH'
res = lemmatizem(query)
query = ' '.join(res)
print(query)

что такое? dwh


In [39]:
chatbot_response(query)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Data Warehouse (DWH) — хранилище, предназначенное для сбора и аналитической обработки исторических данных организации.
Позволяет объединить данные из различных систем и источников в одном месте, а также улучшить качество данных и скорость доступа к ним.
Про DWH в Alpha BI можно узнать по ссылке https://conf.bars.group/pages/viewpage.action?pageId=49586442
